In [22]:
import os
import pandas as pd

def clean_string(s):
    """Limpia el string de espacios adicionales."""
    return s.strip()

def rename_file(old_path, new_path):
    """Renombra un archivo de la ruta old_path a new_path."""
    try:
        os.rename(old_path, new_path)
        print(f"Archivo renombrado de: '{old_path}' a '{new_path}'")
    except Exception as e:
        print(f"Error al renombrar el archivo: {e}")

def search_in_directory(file_name, search_directory):
    """Busca un archivo en un directorio de manera recursiva."""
    for root, dirs, files in os.walk(search_directory):
        if file_name in files:
            return os.path.join(root, file_name)
    return None

def check_mp3_files(library_file):
    """Verifica si todos los archivos MP3 listados en el archivo de la biblioteca existen en el sistema."""
    # Leer todas las hojas del archivo "Biblioteca.xlsx"
    library_df_dict = pd.read_excel(library_file, sheet_name=None)

    missing_files = []
    found_with_space_issue = []
    search_directory = r"E:\Musica-E\DJ Project"  # Directorio donde buscar si no se encuentra el archivo

    # Recorrer todas las hojas de "Biblioteca.xlsx"
    for sheet_name, library_df in library_df_dict.items():
        print(f"Procesando hoja: {sheet_name}")

        if 'Titulo' in library_df.columns and 'Ubicacion' in library_df.columns and 'Nombre' in library_df.columns:
            library_df['Titulo'] = library_df['Titulo'].apply(lambda x: x.replace("’", "'").replace(".mp3", ""))
            library_df['Nombre'] = library_df['Nombre'].apply(lambda x: x.replace("’", "'").replace(".mp3", ""))

            # Verificar la existencia de cada archivo MP3
            for index, row in library_df.iterrows():
                file_path = os.path.join(row['Ubicacion'], row['Nombre'] + ".mp3")
                if not os.path.exists(file_path):
                    # Si el archivo con 'Nombre' no existe, probar con 'Titulo'
                    file_path = os.path.join(row['Ubicacion'], row['Titulo'] + ".mp3")

                # Si todavía no se encuentra el archivo, verificar si existe con un espacio adicional antes de .mp3
                if not os.path.exists(file_path):
                    file_path_with_space = os.path.join(row['Ubicacion'], row['Nombre'] + " .mp3")
                    if os.path.exists(file_path_with_space):
                        # Renombrar el archivo eliminando el espacio adicional
                        new_file_path = os.path.join(row['Ubicacion'], row['Nombre'] + ".mp3")
                        rename_file(file_path_with_space, new_file_path)

                        # Actualizar la entrada en la hoja de cálculo (si es necesario)
                        library_df.at[index, 'Nombre'] = row['Nombre']
                        library_df.at[index, 'Titulo'] = row['Titulo']

                        found_with_space_issue.append({
                            'Nombre': row['Nombre'],
                            'Titulo': row['Titulo'],
                            'Ubicacion': row['Ubicacion'],
                            'File Path': new_file_path
                        })
                    else:
                        # Si no se encuentra, buscar en todo "E:\Musica-E\DJ Project\"
                        search_result = search_in_directory(row['Nombre'] + ".mp3", search_directory)
                        if search_result:
                            new_location = os.path.dirname(search_result)
                            print(f"Archivo encontrado en otra ubicación: {search_result}")
                            
                            # Actualizar la ubicación en la hoja de cálculo
                            library_df.at[index, 'Ubicacion'] = new_location
                            file_path = search_result  # Actualizar la ruta del archivo con la nueva ubicación
                        else:
                            missing_files.append({
                                'Nombre': row['Nombre'],
                                'Titulo': row['Titulo'],
                                'Ubicacion': row['Ubicacion'],
                                'File Path': file_path
                            })
                            print(f"Archivo no encontrado: {file_path}")

        else:
            print(f"La hoja '{sheet_name}' no tiene las columnas necesarias 'Titulo', 'Nombre' y 'Ubicacion'.")

    # Guardar los cambios en el archivo Excel
    with pd.ExcelWriter(library_file, engine='openpyxl') as writer:
        for sheet_name, df in library_df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Archivo Excel actualizado: {library_file}")

    return missing_files, found_with_space_issue

# Configuraciones
library_file = r"E:\Musica-E\Organizacion\Biblioteca.xlsx"

# Verificar la existencia de archivos MP3
missing_files, found_with_space_issue = check_mp3_files(library_file)

# Imprimir los detalles de los archivos faltantes
if missing_files:
    print("\nArchivos MP3 faltantes:")
    for file in missing_files:
        print(f"Nombre: {file['Nombre']}, Título: {file['Titulo']}, Ubicación: {file['Ubicacion']}, Ruta: {file['File Path']}")
else:
    print("Todos los archivos MP3 existen.")

# Imprimir los detalles de los archivos encontrados con un espacio adicional antes de '.mp3' y renombrados
if found_with_space_issue:
    print("\nArchivos que fueron encontrados con un espacio adicional y renombrados:")
    for file in found_with_space_issue:
        print(f"Nombre: {file['Nombre']}, Título: {file['Titulo']}, Ubicación: {file['Ubicacion']}, Ruta: {file['File Path']}")


Procesando hoja: Sheet1
Archivo Excel actualizado: E:\Musica-E\Organizacion\Biblioteca.xlsx
Todos los archivos MP3 existen.
